In [1]:
from trajalign.traj import Traj
from trajalign.average import load_directory
from skimage.measure import profile_line
import matplotlib.pyplot as plt
import numpy as np
import os
import skimage
import pandas as pd
import lmfit
from lmfit.lineshapes import gaussian

In [2]:
def _line_profile_coordinates(src, dst, linewidth=1):
    """Return the coordinates of the profile of an image along a scan line.
    Parameters
    ----------
    src : 2-tuple of numeric scalar (float or int)
        The start point of the scan line.
    dst : 2-tuple of numeric scalar (float or int)
        The end point of the scan line.
    linewidth : int, optional
        Width of the scan, perpendicular to the line
    Returns
    -------
    coords : array, shape (2, N, C), float
        The coordinates of the profile along the scan line. The length of the
        profile is the ceil of the computed length of the scan line.
    Notes
    -----
    This is a utility method meant to be used internally by skimage functions.
    The destination point is included in the profile, in contrast to
    standard numpy indexing.
    """
    src_row, src_col = src = np.asarray(src, dtype=float)
    dst_row, dst_col = dst = np.asarray(dst, dtype=float)
    d_row, d_col = dst - src
    theta = np.arctan2(d_row, d_col)

    length = int(np.ceil(np.hypot(d_row, d_col) + 1))
    # we add one above because we include the last point in the profile
    # (in contrast to standard numpy indexing)
    line_col = np.linspace(src_col, dst_col, length)
    line_row = np.linspace(src_row, dst_row, length)

    # we subtract 1 from linewidth to change from pixel-counting
    # (make this line 3 pixels wide) to point distances (the
    # distance between pixel centers)
    col_width = (linewidth - 1) * np.sin(-theta) / 2
    row_width = (linewidth - 1) * np.cos(theta) / 2
    perp_rows = np.stack([np.linspace(row_i - row_width, row_i + row_width,
                                      linewidth) for row_i in line_row])
    perp_cols = np.stack([np.linspace(col_i - col_width, col_i + col_width,
                                      linewidth) for col_i in line_col])
    return np.stack([perp_rows, perp_cols])

In [3]:
def fusion_frame (m_c):
    for x in m_c:
        n=len(x)   # number of frames
        U=np.empty( ((n-2),2) ) # creation of an empty array of n-2 lines and 2 columns
        
            
        for tau in range ((n-2)):  
# construction of the model for frame = x            
            m1 = np.mean(x.f()[0:tau])   
            m2 = np.mean(x.f()[(tau+1):n])
            m = np.array([m1] * tau + [m2]* (n-tau)) # use np.array because python can't substract list
            e = x.f() - m                           # substract theorical model from data
            U[tau,0] = x.frames()[tau]               # first column of matrix = frame number
            U[tau,1] = np.sum(e**2)                  # second column of matrix = sum of square errors 
        pos= np.argmin( U[:,1])              # get position of frame corresponding to mimimum sum of squared errors
        f_f=x.frames()[pos]              #get frame of fusion
        f_f= f_f-1   #correct for diff frame numeration PT/Fiji
        
    return f_f 

In [4]:
path='/mnt/c/Users/Valentine/Documents/UNIL/projects/ff_prot_mapping/Experiments/Microscopy/spread_signal_detection/Prm1/Rep_1/trajectories/'
Cell=[]
F_frame=[]
path_ev=[]
directory_rep = os.listdir( path)
for folder in directory_rep:
    print(os.path.join(path,folder))
    path_ev.append(os.path.join(path,folder))
    t_l=load_directory(os.path.join(path,folder),pattern= 'Traj',  comment_char = '%' , frames = 0 , coord = ( 1 , 2 ) , f = 3, coord_unit='pxl' )
    Cell.append(t_l)
    m_c = load_directory(os.path.join(path,folder), pattern= '.csv',  comment_char = '#' ,sep= ',', frames = 0 , f = 1 )
    f_f = fusion_frame (m_c)
    F_frame.append(f_f)
    
b_coord=[]
b_t=[]
r_coord=[]
r_t=[]
for traj in Cell:
    for t in traj:
        if '_B' in t.annotations()[ 'file' ]:
            b=t.coord()
            f=t.frames()
            b_coord.append(b)
            b_t.append(f)
        if '_R' in t.annotations()[ 'file' ]:
            r=t.coord()
            r_coord.append(r)
            f=t.frames()
            r_t.append(f)


/mnt/c/Users/Valentine/Documents/UNIL/projects/ff_prot_mapping/Experiments/Microscopy/spread_signal_detection/Prm1/Rep_1/trajectories/cell_000
1-Traj_01_B-01.txt
1-Traj_01_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-mCherry_01.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/UNIL/projects/ff_prot_mapping/Experiments/Microscopy/spread_signal_detection/Prm1/Rep_1/trajectories/cell_001
1-Traj_02_B-01.txt
1-Traj_02_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-mCherry_02.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/UNIL/projects/ff_prot_mapping/Experiments/Microscopy/spread_signal_detection/Prm1/Rep_1/trajectories/cell_002
1-Traj_03_B-01.txt
1-Traj_03_R-03.txt

 >> load_directory: The 'intensity_normalisation' 

/mnt/c/Users/Valentine/Documents/Scripts/notebook/jupyterenv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/c/Users/Valentine/Documents/Scripts/notebook/jupyterenv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-mCherry_33.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/UNIL/projects/ff_prot_mapping/Experiments/Microscopy/spread_signal_detection/Prm1/Rep_1/trajectories/cell_033
1-Traj_34_B-01.txt
1-Traj_34_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-mCherry_34.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/UNIL/projects/ff_prot_mapping/Experiments/Microscopy/spread_signal_detection/Prm1/Rep_1/trajectories/cell_034
1-Traj_35_B-01.txt
1-Traj_35_R-04.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-mCherry_35.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Docume

3-Traj_10_R-06.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

3-mCherry_10.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/UNIL/projects/ff_prot_mapping/Experiments/Microscopy/spread_signal_detection/Prm1/Rep_1/trajectories/cell_063
3-Traj_11_B-01.txt
3-Traj_11_R-03.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

3-mCherry_11.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/UNIL/projects/ff_prot_mapping/Experiments/Microscopy/spread_signal_detection/Prm1/Rep_1/trajectories/cell_064
3-Traj_12_B-02.txt
3-Traj_12_R-03.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

3-mCherry_12.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Use


 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

5-mCherry_03.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/UNIL/projects/ff_prot_mapping/Experiments/Microscopy/spread_signal_detection/Prm1/Rep_1/trajectories/cell_100
5-Traj_04_B-01.txt
5-Traj_04_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

5-mCherry_04.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/UNIL/projects/ff_prot_mapping/Experiments/Microscopy/spread_signal_detection/Prm1/Rep_1/trajectories/cell_101
5-Traj_05_B-01.txt
5-Traj_05_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

5-mCherry_05.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Docume

In [5]:
images={}
i=0
for folder in directory_rep:
    files=os.listdir(os.path.join(path,folder))
    for f in files:
        if f.endswith('tiff'):
            img=skimage.io.imread(os.path.join(path,folder,f))
            images[i]=img
            i+=1

In [6]:
B=[]
R=[]
for i in range(len(b_coord)):
    
    data = pd.DataFrame(b_coord[i]).T
    data['frame']=b_t[i]
    for j in range( 0,images[i].shape[0]):
        if (data.frame==j).sum()==0:
            data =pd.concat([data, pd.DataFrame([None,None,j], index=[0,1,'frame']).T])
    data = data.sort_values('frame').astype(np.float16)
    data = data.reset_index(drop=True)
    data = data.interpolate(axis=0,limit_direction='both')
    B.append(data.iloc[:,0:2])
    
for i in range(len(b_coord)):
    data = pd.DataFrame(r_coord[i]).T
    data['frame']=r_t[i]
    for j in range(0, images[i].shape[0]):
        if (data.frame==j).sum()==0:
            data =pd.concat([data, pd.DataFrame([None,None,j], index=[0,1,'frame']).T])
    data = data.sort_values('frame').astype(np.float16)
    data = data.reset_index(drop=True)
    data = data.interpolate(axis=0,limit_direction='both')
    R.append(data.iloc[:,0:2])

In [7]:
Ext_R=[]
for i in range (len(R)):
    b=B[i]
    r=R[i]
    img=images[i]
    x=[[] for _ in range (len(b[0]))]
    y=[[] for _ in range (len(b[1]))]
    for time in range (len(b[0])):
        n=4
        y[time]=b.iloc[time,0]+n*(r.iloc[time,0]-b.iloc[time,0])
        x[time]=b.iloc[time,1]+n*(r.iloc[time,1]-b.iloc[time,1])    
        while (x[time]>img.shape[2] or x[time]<0 or y[time]>img.shape[1] or y[time]<0):
                n-=1
                if n<0:
                    y[time]=r.iloc[time,0]
                    x[time]=r.iloc[time,1]
                    print ('breaking for i='+str(i)+' at time '+str(time)+' x='+str(x[time])+' y='+ str(y[time]))
                    break
                else:
                    y[time]=b.iloc[time,0]+n*(r.iloc[time,0]-b.iloc[time,0])
                    x[time]=b.iloc[time,1]+n*(r.iloc[time,1]-b.iloc[time,1])
                
           
    C=pd.DataFrame({'y':y,'x':x})    
    Ext_R.append(C)

In [8]:
f_profiles=[]
coord_profiles=[]
# get fluorescence profile + associated coordinates from blue to red
for i in range(0, len(images)):
    img = images[i]
    start=B[i]
    end=Ext_R[i]
    lines=[]
    coords=[]
    
    for time in range(0, F_frame[i]):
        l = profile_line(img[time],start.iloc[time] , end.iloc[time],linewidth=15,reduce_func=None) 
        line=[[] for _ in range (len(l))]
        coord= _line_profile_coordinates(start.iloc[time] , end.iloc[time])
        for k in range (len(l)):
            line[k]=np.mean(l[k])
        lines.append(line)
        coords.append(coord)
        
    coord_profiles.append(coords)
    f_profiles.append(lines)

In [9]:
Centroid_xyfit=[]
g_coord_xyfit=[]
for index in range(0, len(images)):
    print(index)
    img=images[index]
    t=Traj()
    x_center=[]
    y_center=[]
    f_center=[]
    frame=[]
    for time in range(0, F_frame[index]):
        X=coord_profiles[index][time][0].reshape((len(coord_profiles[index][time][0]),))
        Y=coord_profiles[index][time][1].reshape((len(coord_profiles[index][time][1]),))
        F=f_profiles[index][time]
        df=pd.DataFrame({'coord_x': X, 'coord_y': Y, 'fluo': F})
        
        
        
        y=df.coord_y
        x=df.coord_x
        f=df.fluo
        model_x = lmfit.models.GaussianModel()
        pars_x = model_x.guess(f,x)
        result_x = model_x.fit(f, x=x, params=pars_x)
        model_y = lmfit.models.GaussianModel()
        pars_y = model_y.guess(f,y)
        result_y = model_y.fit(f, x=y, params=pars_y)
        if result_y.rsquared>0.6 or result_x.rsquared>0.6:
            x_m=result_x.params['center'].value
            y_m=result_y.params['center'].value
            f_m=result_x.params['height'].value
        else: 
            x_m=np.nan
            y_m=np.nan
            f_m=np.nan
         
        
        frame.append(time)
        f_center.append(f_m)
        x_center.append(x_m)
        y_center.append(y_m)
    
    t.input_values('frames',frame)
    t.input_values('coord', [x_center,y_center], unit='pixels')
    t.input_values('f', f_center)
    t.save(os.path.join(path_ev[index], 'Traj_G.txt'))
    Centroid_xyfit.append(t)
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
